<a href="https://colab.research.google.com/github/neelmehta00/RealEstateMoguls/blob/master/Official_Real_Estate_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importing models, functions to be used

In [ ]:
monthly_avg = [393.4, 395.5, 395.5, 398, 401, 404.6, 407.9, 412.9, 414.9, 416.9, 416.9, 418.2, 419.7, 423.1, 424.6, 424.6, 426.8, 430, 430, 430.8, 431.9, 433, 434.4, 436.8, 438, 442.1, 442.1, 444.3, 444.9, 445.2, 448.7, 451.2, 452.4, 452.4, 455.6, 458.2, 459.4, 458.7, 458.4, 458.4, 458.7, 460.7, 461.3, 462.3, 462.7, 464.2, 465.3, 465.2, 464.1, 462.9, 462.7, 463.2, 464.1, 464.2, 464.1, 463.7, 463.4, 463, 462.5, 462.7, 464.4, 465.6, 465.6, 461.3, 457.8, 455.8, 454.3, 453.1, 452.5, 452.1, 452.1, 452]
import pandas as pd
import math
import random
import scipy.stats as stats
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
R = 6373.0

#This is a function for distance(km) between two points given their lat and longs. 
def LatLongDist(lat1, long1, lat2, long2):
  A = math.radians(lat1)
  B = math.radians(long1)
  C = math.radians(lat2)
  D = math.radians(long2)
  dlon = D - B
  dlat = C - A
  a = math.sin(dlat / 2)**2 + math.cos(A) * math.cos(C) * math.sin(dlon / 2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  return R * c

#Below is non-Traditional scoring function. 

def score_from_distances(house_to_stores, d1,d2,d3,d4,d5):
  count1 = 0
  count2 = 0
  count3 = 0
  count5 = 0 
  count7 = 0
  score = 10
  for dist in house_to_stores:
    if dist <= d1:
      count1 += 1
    if dist <= d2:
      count2 += 1
    if dist <= d3:
      count3 += 1
    if dist <= d4:
      count5 += 1
    if dist <= d5:
      count7 += 1
  if count1 >= 2:
    pass
  elif count1 >= 1:
    score -= 1
  elif count2 >= 2:
    score -= 2
  elif count2 >= 1:
    score -= 3
  elif count3 >= 2:
    score -= 4
  elif count3 >= 1:
    score -= 5
  elif count5 >= 2:
    score -= 6
  elif count5 >= 1:
    score -= 7
  elif count7 >= 2:
    score -= 8
  elif count7 >= 1:
    score -= 9
  else: score = 0
  return score

#Below is the atypical scoring function for inferior stores (fastfood)
def score_from_inferiors(house_to_stores, d1,d2,d3,d4,d5):
  count1 = 0
  count2 = 0
  count3 = 0
  count4 = 0 
  count5 = 0
  score = 10
  for dist in house_to_stores:
    if dist > d5:
      count1 += 1
    if dist >= d4:
      count2 += 1
    if dist >= d3:
      count3 += 1
    if dist >= d2:
      count4 += 1
    if dist >= d1:
      count5 += 1
  if count1 == 1:
    pass
  elif count1 >= 2:
    score -= 1
  elif count2 == 1:
    score -= 2
  elif count2 >= 2:
    score -= 3
  elif count3 == 1:
    score -= 4
  elif count3 >= 2:
    score -= 5
  elif count4 == 1:
    score -= 6
  elif count4 >= 2:
    score -= 7
  elif count5 == 1:
    score -= 8
  elif count5 >= 2:
    score -= 9
  else: score = 0
  return score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2. Preprocessing Houses(1/3)



In [ ]:
house_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/NewZest.csv')
print(len(house_df))
address = house_df[" Address"]
price = house_df[" Price"]
link = house_df[" link"]
link1 = house_df[" link1"]
listdrop1 = []


#FIRST, getting rid of duplicate houses
for i in range(len(house_df)): 
  for j in range(len(house_df)):
    if i != j:
      if address.iloc[i] == address.iloc[j] and price.iloc[i] == price.iloc[j]:
        listdrop1.append(i)


#SECOND, house from 1before 2015, 2useless zestimates, 3price is nan
for i in range(len(house_df)):
    if not(house_df[' price2'].iloc[i] > 0 and (int(house_df[' date2'].iloc[i][-4:]) >= 2015)): # from before 2015
      listdrop1.append(i)
    if house_df[' ZestYear 2015'].iloc[i] == "Today" or house_df[' ZestYear 2015'].iloc[i] == "Listed for": # useless zestimates
      listdrop1.append(i)
    if math.isnan(house_df[' ZestPrice 2020'].iloc[i]) or math.isnan(house_df[' ZestPrice 2015 '].iloc[i]): #nan
      listdrop1.append(i)

      
dictOfYear = {}
listdrop2 = []
for i in range(len(listdrop1)):
  year = listdrop1[i]
  dictOfYear[year] = listdrop1[i]
for key in dictOfYear:
  listdrop2.append(key)


for item in listdrop2:
  house_df.drop([item], axis=0, inplace=True)
print(len(house_df))

511
185


# Preprocessing Houses (2/3)

In [ ]:
#THIRD, get final_date, final_price, index_list, initial_date, initial_price, and monthly_change

index_list = []
final_date = []
final_price = []
absolute_list = []
lat_list = []
long_list = []
count = 0
for index, row in house_df.iterrows():
  index_list.append(index)
  #COME BACK TO THIS!!!!!!!!! (need to account for 2 digit months)
  
  
  if (int(row[" date1"][0])) < 5:
    final_date.append(row[' date1'])
    final_price.append(row[' price1'])
  else:
    if int(row[' price1']) < int(row[' price2']) and int(row[' date2'][-4:]) == 2020 and ((int(row[" date1"][0])) - (int(row[" date2"][0])) < 6):
      count = count + 1
      final_date.append(row[' date2'])
      final_price.append(row[' price2'])
    else:
      final_date.append(row[' date1'])
      final_price.append(row[' price1'])

  absolute_list.append(row['absolute'])
  lat_list.append(row[' Latitude'])
  long_list.append(row[' Longitude'])
print("# houses affected by corona: ", count)

initial_date = []
initial_price = []
count = 0
for index, row in house_df.iterrows():
  count = count + 1
  if (row[' price7'] > 0) and (int(row[' date7'][-4:]) >= 2015):
    initial_date.append(row[' date7'])
    initial_price.append(row[' price7'])
  elif (row[' price6'] > 0) and (int(row[' date6'][-4:]) >= 2015):
    initial_date.append(row[' date6'])
    initial_price.append(row[' price6'])
  elif (row[' price5'] > 0) and (int(row[' date5'][-4:]) >= 2015):
    initial_date.append(row[' date5'])
    initial_price.append(row[' price5'])
  elif (row[' price4'] > 0) and (int(row[' date4'][-4:]) >= 2015):
    initial_date.append(row[' date4'])
    initial_price.append(row[' price4'])
  elif (row[' price3'] > 0) and (int(row[' date3'][-4:]) >= 2015):
    initial_date.append(row[' date3'])
    initial_price.append(row[' price3'])
  else:
    initial_date.append(row[' date2'])
    initial_price.append(row[' price2'])


print("house count after first processing: %d"%len(initial_price))
print("")

#getting rid of same year/month
year_one = house_df[" ZestPrice 2015 "]
year2015 = []
for i in range(len(year_one)):
  year2015.append(year_one.iloc[i])


year_six = house_df[' ZestPrice 2020']
year2020 = []
for i in range(len(year_six)):
  year2020.append(year_six.iloc[i])

index_list2 = []
for i in range(len(initial_date)):
  if initial_date[i][-4:] == final_date[i][-4:] and initial_date[i][:2] == final_date[i][:2]:
    index_list2.append(i)

to_subtract = 0

while len(index_list2) > 0:
  ind = index_list2.pop(0) - to_subtract
  del initial_date[ind]
  del initial_price[ind]
  del final_date[ind]
  del final_price[ind]
  del absolute_list[ind]
  del lat_list[ind]
  del long_list[ind]
  del year2015[ind]
  del year2020[ind]

  to_subtract += 1

print("house count after getting rid of same year/month: %d"%len(final_price))
print("")

#getting monthly change list, the target

#mount pleasant average prices
monthly_change = []
for i in range(len(initial_date)):
  try:
    initial = (int(initial_date[i][-4:]) - 2015) * 12 + int(initial_date[i][:2])
  except:
    initial = (int(initial_date[i][-4:]) - 2015) * 12 + int(initial_date[i][:1])

  try:
    final = (int(final_date[i][-4:]) - 2015) * 12 + int(final_date[i][:2])
  except:
    final = (int(final_date[i][-4:]) - 2015) * 12 + int(final_date[i][:1])
  num_month = final - initial
  MP_avg = (((monthly_avg[final]/monthly_avg[initial])**(1/(num_month))) - 1) * 100
  house_avg = (((final_price[i]/initial_price[i])**(1/num_month) - 1) * 100)
  monthly_change.append(house_avg - MP_avg)

print("monthly_change: ", monthly_change)
print("monthly change length: %d" %len(monthly_change))
data = {'house_absolute': absolute_list,
        'initial date': initial_date,
        'initial price': initial_price,
        'final date': final_date,
        'final price': final_price,
        'monthly change': monthly_change,
        'latitude' : lat_list,
        'longitude' : long_list,
        'year2015': year2015,
        'year2020':year2020
        }
house_df2 = pd.DataFrame(data, columns = ['house_absolute', 'initial date', 'initial price', 'final date', 'final price', 'monthly change', 'latitude', 'longitude',  'year2015', 'year2020'])

# houses affected by corona:  101
house count after first processing: 185

house count after getting rid of same year/month: 154

monthly_change:  [0.007383271526562041, -0.8376117565893404, -0.17386458465421528, 3.9379797435336927, -0.246627219525275, -0.6038118078849575, -1.7463799912242228, -0.2904913076321236, -0.715954961096843, 0.6325177355252398, 0.12325365221407258, 1.165890780747092, 0.051945831253052965, -1.8403583094156795, 2.0716351441732117, 1.1898271283025563, -0.44258824964931165, -0.25737272109841447, -0.8145536105775841, 3.746197153551556, 0.13242109909512578, -0.1404644817322831, 0.03941098394486531, 10.020478645195608, -0.05636549614976838, 0.7850668832750096, -0.04882840384524689, 0.7587253414264028, 0.36781692669478616, -0.698873068058381, 0.4729466701342666, -0.26959152382247664, 32.04639047958048, -0.0235560375539956, -0.413158374505429, 0.666786490033866, -3.4429936002327244, 0.3690525261574873, -0.014969354878524577, -0.1562738881686565, 0.1593809476351682, -0.

# Preprocessing Houses (3/3)

In [ ]:
#THIS IS FOR STATISTICALLY DEFINED OUTLEIRS, BUT THAT LEADS TO TOO MANY OF THEM

monthly_change2 = monthly_change
monthly_change2.sort()
Q1 = monthly_change2[29]
Q3 = monthly_change2[87]
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

listdrop2 = []
for index, row in house_df2.iterrows():
  if row['monthly change'] > upper_bound or row['monthly change'] < lower_bound:
    listdrop2.append(index)
print("upper: %f"%upper_bound)
print("lower: %f"%lower_bound)

print("number of statistical outliers in terms of monthly change: %d"%len(listdrop2))


for item in listdrop2:
  house_df2.drop([item], axis=0, inplace=True)

print("house count after getting rid of outliers: %d" % len(house_df2))

upper: 1.264234
lower: -1.126212
number of statistical outliers in terms of monthly change: 38
house count after getting rid of outliers: 116


# 3. Preparing Atypical data and scores for houses



In [ ]:
grocery_df = pd.read_csv("https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20grocery.csv")
coffeeshop_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20Coffee%20Shops.csv')
fastfood_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20FastFood.csv')
parks_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20Parks.csv')
gas_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20gas.csv')
theater_df = pd.read_csv('https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20Movie%20Theater.csv')
wholefoods_df = pd.read_csv("https://raw.githubusercontent.com/neelmehta00/RealEstateMoguls/master/Real%20Estate%20-%20WholeFoods.csv")

grocery_latlist = grocery_df['Latitude']
grocery_longlist = grocery_df['Longitude']
coffeeshop_latlist = coffeeshop_df['Latitude']
coffeeshop_longlist = coffeeshop_df['Longitude']
fastfood_latlist = fastfood_df['Latitude']
fastfood_longlist = fastfood_df['Longitude']
parks_latlist = parks_df['Latitude']
parks_longlist = parks_df['Longitude']
gas_latlist = gas_df['Latitude']
gas_longlist = gas_df['Longitude']
theater_latlist = theater_df['Latitude']
theater_longlist = theater_df['Longitude']
wholefoods_latlist = wholefoods_df['Latitude']
wholefoods_longlist = wholefoods_df['Longitude']

house_latlist = house_df2['latitude']
house_longlist = house_df2['longitude']

categories_latlist = [grocery_latlist, coffeeshop_latlist, fastfood_latlist, parks_latlist, gas_latlist, theater_latlist, wholefoods_latlist]
categories_longlist = [grocery_longlist, coffeeshop_longlist, fastfood_longlist, parks_longlist, gas_longlist, theater_longlist, wholefoods_longlist]
num_categories = len(categories_latlist)
num_houses = len(house_df2)
big_list = []

grocscorelist = []
cofscorelist = []
ffscorelist = []
parkscorelist = []
gasscoreslist = []
theaterscorelist = []
wfscorelist = []

for i in range(num_houses):
  house_list = []
  for j in range(num_categories):
    num_stores = len(categories_latlist[j])
    dist_to_stores = [LatLongDist(float(house_latlist.iloc[i]), float(house_longlist.iloc[i]), float(categories_latlist[j].iloc[a]), float(categories_longlist[j].iloc[a])) for a in range(num_stores)]

    if j == 0: #grocery
      house_list.append(score_from_distances(dist_to_stores, 0.03965, .4588, 1.28916, 3.94706,4.63319))
      grocscorelist.append(score_from_distances(dist_to_stores, 0.03965, .4588, 1.28916, 3.94706,4.63319))
    elif j == 1: #coffeeshop
      house_list.append(score_from_distances(dist_to_stores, .24646, .623074, 1.56503, 3.42349,4.01676))
      cofscorelist.append(score_from_distances(dist_to_stores, .24646, .623074, 1.56503, 3.42349,4.01676))
    elif j == 2: #fastfood
      house_list.append(score_from_inferiors(dist_to_stores, 0.45706, 0.807129, 2.9667, 3.87406, 4.70226))
      ffscorelist.append(score_from_inferiors(dist_to_stores, 0.45706, 0.807129, 2.9667, 3.87406, 4.70226))
    elif j == 3: #parks
      house_list.append(score_from_distances(dist_to_stores, .18968, .60710, 1.89989, 3.56676,4.11641))
      parkscorelist.append(score_from_distances(dist_to_stores, .18968, .60710, 1.89989, 3.56676,4.11641))
    elif j == 4: #gas
      house_list.append(score_from_distances(dist_to_stores, .59958, 1.59736, 2.90703, 3.33313,4.97810))
      gasscoreslist.append(score_from_distances(dist_to_stores, .59958, 1.59736, 2.90703, 3.33313,4.97810))
    elif j == 5: #theater
      house_list.append(score_from_distances(dist_to_stores, .31673, 1.9024, 2.9417,3.61168, 4.71139))
      theaterscorelist.append(score_from_distances(dist_to_stores, .31673, 1.9024, 2.9417,3.61168, 4.71139))
    elif j == 6: #wholefoods
      house_list.append(score_from_distances(dist_to_stores, .3122, .9226, 1.29546, 1.6959,2.9939))
      wfscorelist.append(score_from_distances(dist_to_stores, .3122, .9226, 1.29546, 1.6959,2.9939))
  big_list.append(house_list)

print(len(wfscorelist))
print(len(grocscorelist))

HTTPError: ignored

# 4. Preparing Typical data

In [ ]:
#list of annul growth rate for zest
#percent change for zestimate
year_1 = house_df2["year2015"]
year_6 = house_df2["year2020"]


biglist1 = []
for i in range(len(house_df2)):
  biglist1.append((((year_6.iloc[i]/year_1.iloc[i])**(0.2)) - 1) * 100)

print(len(biglist1))

In [ ]:
#list of annul growth rate for zest
#percent change for zestimate
year_1 = house_df2["year2015"]
year_6 = house_df2["year2020"]

avgmpch = 3.378

biglist1 = []
for i in range(len(house_df2)):
  biglist1.append((((year_6.iloc[i]/year_1.iloc[i])**(0.2)) - 1) * 100)

listB = [(biglist1[i] - avgmpch) for i in range(len(biglist1))]
listB_np = np.array(listB)
dez = stats.zscore(listB_np)

nonavgscore = [] 
tempnonavgscore =[]
listB = []
for i in range(len(biglist1)):

    comp = dez[i]
    score = 0
    if(comp <= -4):
      score = 1
    elif(comp <= -3):
      score = -6
    elif(comp <= -2):
      score = -3
    elif(comp <= -1):
      score = -1
    elif(comp <= -0.5):
      score = -0.5
    elif(comp <= 0.5):
      score = 0.5
    elif(comp <= 1):
      score = 1
    elif(comp <= 2):
      score = 3
    elif(comp <= 3):
      score = 6
    elif(comp <= 4):
      score = 10
    else:
      score = 15
    tempnonavgscore.append(score)
    listB.append(comp)

    
traditionalscores = tempnonavgscore
for i in range(len(big_list)):
  big_list[i].append(tempnonavgscore[i])

# 5. Running the Model

In [ ]:
#first define accuracy and prediction funtions
def get_accuracy(margin, actualList, predList):
  length = len(actualList)
  correct = 0
  for i in range(length):
    if abs(actualList[i] - predList[i]) <= margin:
      correct += 1
  return (correct / length * 100)
def predict(scores_from_categories, intercept, coef_list):
    returnVal = 0
    for i in range(len(scores_from_categories)):
      returnVal += scores_from_categories[i] * coef_list[i]
    return (returnVal + intercept)

score = 0
coef = [0, 0, 0, 0, 0, 0, 0, 0]    
for i in range(10000):  
  X = big_list
  Y = []
  for target in house_df2['monthly change']:
    Y.append(target)
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

  regr = linear_model.LinearRegression()
  regr.fit(x_train, y_train)
  
  pred_list = []
  for score_list in x_test:
    pred_list.append(predict(score_list, regr.intercept_, regr.coef_))
  
  score += get_accuracy(0.4, y_test, pred_list)
  for i in range(len(coef)):
    coef[i] += regr.coef_[i]
print("Accuray: %f"%(score/10000)) 
for i in range(len(coef)):
  coef[i] = coef[i]/10000
print(coef) 

In [ ]:
seattle_monthly = []